In [1]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

# Set the directories for training and testing images
train_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Train'
test_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Test'

# Load the training CSV file with image labels and percentages
train_csv_file = 'train.csv'
train_data = pd.read_csv(train_csv_file, header=None, names=['image_filename', 'percentage_covid'])

# Load the testing CSV file with image labels and percentages
test_csv_file = 'test.csv'
test_data = pd.read_csv(test_csv_file, skiprows=1, header=None, names=['image_filename', 'percentage_covid'])

# Convert the percentage values from string to float
train_data['percentage_covid'] = train_data['percentage_covid'].astype(float)
test_data['percentage_covid'] = test_data['percentage_covid'].astype(float)

# Create the ImageDataGenerator for data preprocessing
datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training set images
train_set = datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=train_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    shuffle=True
)

# Load and preprocess the testing set images
test_set = datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other'
)

# Define the checkpoint filepath
checkpoint_filepath = 'final2.h5'

# Create the ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# Define the CNN model
CNN = Sequential()
CNN.add(Convolution2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Convolution2D(64, (3, 3), activation="relu"))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Convolution2D(128, (3, 3), activation="relu"))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Flatten())
CNN.add(Dense(units=256, activation="relu"))
CNN.add(Dropout(0.5))
CNN.add(Dense(units=128, activation="relu"))
CNN.add(Dropout(0.5))
CNN.add(Dense(units=1, activation="linear"))

# Compile the CNN model with a lower learning rate
learning_rate = 0.0001
optimizer = Adam(lr=learning_rate)
CNN.compile(optimizer=optimizer, loss="mean_squared_error")

# Train the CNN model with the checkpoint callback
history = CNN.fit(
    train_set,
    epochs=60,
    validation_data=test_set,
    callbacks=[checkpoint]
)


Found 2859 validated image filenames.
Found 1127 validated image filenames.


C:\Users\Menna Fawzy\myenv\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/60
90/90 [==============================] - ETA: 0s - loss: 773.0065
Epoch 1: val_loss improved from inf to 426.09943, saving model to final2.h5
90/90 [==============================] - 296s 3s/step - loss: 773.0065 - val_loss: 426.0994
Epoch 2/60
90/90 [==============================] - ETA: 0s - loss: 618.0958
Epoch 2: val_loss improved from 426.09943 to 271.99942, saving model to final2.h5
90/90 [==============================] - 292s 3s/step - loss: 618.0958 - val_loss: 271.9994
Epoch 3/60
90/90 [==============================] - ETA: 0s - loss: 440.5201
Epoch 3: val_loss improved from 271.99942 to 217.86555, saving model to final2.h5
90/90 [==============================] - 196s 2s/step - loss: 440.5201 - val_loss: 217.8656
Epoch 4/60
90/90 [==============================] - ETA: 0s - loss: 387.0218
Epoch 4: val_loss improved from 217.86555 to 186.12639, saving model to final2.h5
90/90 [==============================] - 185s 2s/step - loss: 387.0218 - val_loss: 186.1264
Ep

In [ ]:
#best model
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error

# Set the directories for training and testing images
train_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Train'
test_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Test'

# Load the training CSV file with image labels and percentages
train_csv_file = 'train.csv'
train_data = pd.read_csv(train_csv_file, header=None, names=['image_filename', 'percentage_covid'])

# Load the testing CSV file with image labels and percentages
test_csv_file = 'test.csv'
test_data = pd.read_csv(test_csv_file, skiprows=1, header=None, names=['image_filename', 'percentage_covid'])

# Convert the percentage values from string to float
train_data['percentage_covid'] = train_data['percentage_covid'].astype(float)
test_data['percentage_covid'] = test_data['percentage_covid'].astype(float)

# Create the ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training set images
train_set = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=train_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    shuffle=True
)

# Load and preprocess the testing set images
test_set = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    shuffle=False  # Set shuffle to False to preserve the order of images
)

# Define the checkpoint and early stopping callbacks
checkpoint_filepath = 'final9.h5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=25,
    mode='min',
    restore_best_weights=True
)

# Define the CNN model
CNN = Sequential()
CNN.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(128, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(256, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(512, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Flatten())
CNN.add(Dense(512, activation='relu'))
CNN.add(Dropout(0.5))
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.5))
CNN.add(Dense(1, activation='linear'))

# Compile the CNN model with Adam optimizer
optimizer = Adam(learning_rate=0.0001)
CNN.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the CNN model with the checkpoint and early stopping callbacks
history = CNN.fit(
    train_set,
    epochs=150,  # Increase the number of epochs
    validation_data=test_set,
    callbacks=[checkpoint, early_stopping]
)

# Load the best model weights
CNN.load_weights(checkpoint_filepath)

# Evaluate the model on the testing set
predictions = CNN.predict(test_set)
mse = mean_squared_error(test_data['percentage_covid'], predictions)
print("MSE on the testing set:", mse)


Found 2859 validated image filenames.
Found 1127 validated image filenames.
Epoch 1/150
90/90 [==============================] - ETA: 0s - loss: 775.7809
Epoch 1: val_loss improved from inf to 444.99246, saving model to final9.h5
90/90 [==============================] - 908s 10s/step - loss: 775.7809 - val_loss: 444.9925
Epoch 2/150
90/90 [==============================] - ETA: 0s - loss: 723.1884
Epoch 2: val_loss improved from 444.99246 to 385.61496, saving model to final9.h5
90/90 [==============================] - 713s 8s/step - loss: 723.1884 - val_loss: 385.6150
Epoch 3/150
90/90 [==============================] - ETA: 0s - loss: 620.0771
Epoch 3: val_loss improved from 385.61496 to 236.27615, saving model to final9.h5
90/90 [==============================] - 615s 7s/step - loss: 620.0771 - val_loss: 236.2762
Epoch 4/150
90/90 [==============================] - ETA: 0s - loss: 512.7646
Epoch 4: val_loss did not improve from 236.27615
90/90 [==============================] - 542s 

In [1]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error

# Set the directories for training and testing images
train_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Train'
test_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Test'

# Load the training CSV file with image labels and percentages
train_csv_file = 'train.csv'
train_data = pd.read_csv(train_csv_file, header=None, names=['image_filename', 'percentage_covid'])

# Load the testing CSV file with image labels and percentages
test_csv_file = 'test.csv'
test_data = pd.read_csv(test_csv_file, skiprows=1, header=None, names=['image_filename', 'percentage_covid'])

# Convert the percentage values from string to float
train_data['percentage_covid'] = train_data['percentage_covid'].astype(float)
test_data['percentage_covid'] = test_data['percentage_covid'].astype(float)

# Create the ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training set images
train_set = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=train_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(150, 150),
    batch_size=32,
    class_mode='other',
    shuffle=True
)

# Load and preprocess the testing set images
test_set = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(150, 150),
    batch_size=32,
    class_mode='other',
    shuffle=False  # Set shuffle to False to preserve the order of images
)

# Define the checkpoint and early stopping callbacks
checkpoint_filepath = 'final9.h5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=25,
    mode='min',
    restore_best_weights=True
)

# Define the CNN model
CNN = Sequential()
CNN.add(Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(128, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(256, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(512, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Flatten())
CNN.add(Dense(512, activation='relu'))
CNN.add(Dropout(0.5))
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.5))
CNN.add(Dense(1, activation='linear'))

# Compile the CNN model with Adam optimizer
optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate
CNN.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the CNN model with the checkpoint and early stopping callbacks
history = CNN.fit(
    train_set,
    epochs=200,  # Increase the number of epochs
    validation_data=test_set,
    callbacks=[checkpoint, early_stopping]
)

# Load the best model weights
CNN.load_weights(checkpoint_filepath)

# Evaluate the model on the testing set
predictions = CNN.predict(test_set)
mse = mean_squared_error(test_data['percentage_covid'], predictions)
print("MSE on the testing set:", mse)


Found 2859 validated image filenames.
Found 1127 validated image filenames.
Epoch 1/200
90/90 [==============================] - ETA: 0s - loss: 794.4016
Epoch 1: val_loss improved from inf to 453.42114, saving model to final9.h5
90/90 [==============================] - 769s 8s/step - loss: 794.4016 - val_loss: 453.4211
Epoch 2/200
90/90 [==============================] - ETA: 0s - loss: 737.0858
Epoch 2: val_loss improved from 453.42114 to 433.14911, saving model to final9.h5
90/90 [==============================] - 755s 8s/step - loss: 737.0858 - val_loss: 433.1491
Epoch 3/200
90/90 [==============================] - ETA: 0s - loss: 703.2331
Epoch 3: val_loss improved from 433.14911 to 376.56448, saving model to final9.h5
90/90 [==============================] - 727s 8s/step - loss: 703.2331 - val_loss: 376.5645
Epoch 4/200
90/90 [==============================] - ETA: 0s - loss: 585.3768
Epoch 4: val_loss improved from 376.56448 to 169.10045, saving model to final9.h5
90/90 [=======

In [2]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.metrics import mean_squared_error
import math

# Set the directories for training and testing images
train_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Train'
test_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Test'

# Load the training CSV file with image labels and percentages
train_csv_file = 'train.csv'
train_data = pd.read_csv(train_csv_file, header=None, names=['image_filename', 'percentage_covid'])

# Load the testing CSV file with image labels and percentages
test_csv_file = 'test.csv'
test_data = pd.read_csv(test_csv_file, skiprows=1, header=None, names=['image_filename', 'percentage_covid'])

# Convert the percentage values from string to float
train_data['percentage_covid'] = train_data['percentage_covid'].astype(float)
test_data['percentage_covid'] = test_data['percentage_covid'].astype(float)

# Create the ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training set images
train_set = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=train_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    shuffle=True
)

# Load and preprocess the testing set images
test_set = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    shuffle=False  # Set shuffle to False to preserve the order of images
)

# Define the checkpoint and early stopping callbacks
checkpoint_filepath = 'final20.h5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=25,
    mode='min',
    restore_best_weights=True
)

# Define the learning rate schedule using the cosine annealing schedule
def cosine_annealing(epoch, lr):
    epochs = 100
    max_lr = 0.001
    min_lr = 0.00001
    return min_lr + (max_lr - min_lr) * (1 + math.cos(math.pi * epoch / epochs)) / 2

# Define the CNN model
CNN = Sequential()
CNN.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(128, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(256, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(512, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Flatten())
CNN.add(Dense(512, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(1, activation='linear'))

# Compile the CNN model with Adam optimizer and learning rate schedule
optimizer = Adam(learning_rate=0.0001)
CNN.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the CNN model with the checkpoint, early stopping, and learning rate scheduler callbacks
history = CNN.fit(
    train_set,
    epochs=100,  # Increase the number of epochs
    validation_data=test_set,
    callbacks=[checkpoint, early_stopping, LearningRateScheduler(cosine_annealing)]
)

# Load the best model weights
CNN.load_weights(checkpoint_filepath)

# Evaluate the model on the testing set
predictions = CNN.predict(test_set)
mse = mean_squared_error(test_data['percentage_covid'], predictions)
print("MSE on the testing set:", mse)


Found 2859 validated image filenames.
Found 1127 validated image filenames.
Epoch 1/100
90/90 [==============================] - ETA: 0s - loss: 969.2651 
Epoch 1: val_loss improved from inf to 477.77042, saving model to final20.h5
90/90 [==============================] - 1425s 16s/step - loss: 969.2651 - val_loss: 477.7704 - lr: 0.0010
Epoch 2/100
90/90 [==============================] - ETA: 0s - loss: 760.3798 
Epoch 2: val_loss improved from 477.77042 to 448.84433, saving model to final20.h5
90/90 [==============================] - 1437s 16s/step - loss: 760.3798 - val_loss: 448.8443 - lr: 9.9976e-04
Epoch 3/100
90/90 [==============================] - ETA: 0s - loss: 756.5435 
Epoch 3: val_loss did not improve from 448.84433
90/90 [==============================] - 1164s 13s/step - loss: 756.5435 - val_loss: 464.4565 - lr: 9.9902e-04
Epoch 4/100
90/90 [==============================] - ETA: 0s - loss: 713.5294 
Epoch 4: val_loss improved from 448.84433 to 311.97379, saving model t

Epoch 34/100
90/90 [==============================] - ETA: 0s - loss: 159.8800
Epoch 34: val_loss did not improve from 47.88658
90/90 [==============================] - 544s 6s/step - loss: 159.8800 - val_loss: 109.2523 - lr: 7.5698e-04
Epoch 35/100
90/90 [==============================] - ETA: 0s - loss: 145.9189
Epoch 35: val_loss did not improve from 47.88658
90/90 [==============================] - 397s 4s/step - loss: 145.9189 - val_loss: 72.7747 - lr: 7.4347e-04
Epoch 36/100
90/90 [==============================] - ETA: 0s - loss: 151.3124
Epoch 36: val_loss did not improve from 47.88658
90/90 [==============================] - 402s 4s/step - loss: 151.3124 - val_loss: 118.4253 - lr: 7.2973e-04
Epoch 37/100
90/90 [==============================] - ETA: 0s - loss: 135.1124
Epoch 37: val_loss did not improve from 47.88658
90/90 [==============================] - 403s 4s/step - loss: 135.1124 - val_loss: 123.4640 - lr: 7.1576e-04
Epoch 38/100
90/90 [==============================] -

In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.metrics import mean_squared_error
import math

# Set the directories for training and testing images
train_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Train'
test_image_dir = 'C:\\Users\\Menna Fawzy\\Documents\\Computer Vision AAST\\computer vision\\covid-final-exam\\Test'

# Load the training CSV file with image labels and percentages
train_csv_file = 'train.csv'
train_data = pd.read_csv(train_csv_file, header=None, names=['image_filename', 'percentage_covid'])

# Load the testing CSV file with image labels and percentages
test_csv_file = 'test.csv'
test_data = pd.read_csv(test_csv_file, skiprows=1, header=None, names=['image_filename', 'percentage_covid'])

# Convert the percentage values from string to float
train_data['percentage_covid'] = train_data['percentage_covid'].astype(float)
test_data['percentage_covid'] = test_data['percentage_covid'].astype(float)

# Create the ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training set images
train_set = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=train_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    shuffle=True
)

# Load and preprocess the testing set images
test_set = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_image_dir,
    x_col='image_filename',
    y_col='percentage_covid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='other',
    shuffle=False  # Set shuffle to False to preserve the order of images
)

# Define the checkpoint and early stopping callbacks
checkpoint_filepath = 'final20.h5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,
    mode='min',
    restore_best_weights=True
)



# Define the CNN model
CNN = Sequential()
CNN.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(128, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(256, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Conv2D(512, (3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Flatten())
CNN.add(Dense(512, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.2))
CNN.add(Dense(1, activation='linear'))

# Compile the CNN model with Adam optimizer and learning rate schedule
optimizer = Adam(learning_rate=0.001)
CNN.compile(optimizer=optimizer, loss='mean_squared_error')

# Load the weights of the model from the H5 file if it exists
try:
    CNN.load_weights(checkpoint_filepath)
    print("Model weights loaded from H5 file.")
except (OSError, IOError):
    print("No H5 file found. Starting training from scratch.")

# Train the CNN model with the checkpoint, early stopping, and learning rate scheduler callbacks
history = CNN.fit(
    train_set,
    epochs=100,  # Increase the number of epochs
    initial_epoch=history.epoch[-1]+1 if 'history' in locals() else 0,  # Continue from the last completed epoch if available
    validation_data=test_set,
    callbacks=[checkpoint, early_stopping]
)

# Load the best model weights
CNN.load_weights(checkpoint_filepath)

# Evaluate the model on the testing set
predictions = CNN.predict(test_set)
mse = mean_squared_error(test_data['percentage_covid'], predictions)
print("MSE on the testing set:", mse)


Found 2859 validated image filenames.
Found 1127 validated image filenames.
Model weights loaded from H5 file.
Epoch 1/100
90/90 [==============================] - ETA: 0s - loss: 220.0367
Epoch 1: val_loss improved from inf to 94.77261, saving model to final20.h5
90/90 [==============================] - 408s 5s/step - loss: 220.0367 - val_loss: 94.7726
Epoch 2/100
90/90 [==============================] - ETA: 0s - loss: 179.7141
Epoch 2: val_loss improved from 94.77261 to 75.76309, saving model to final20.h5
90/90 [==============================] - 406s 5s/step - loss: 179.7141 - val_loss: 75.7631
Epoch 3/100
90/90 [==============================] - ETA: 0s - loss: 180.0580
Epoch 3: val_loss improved from 75.76309 to 61.94953, saving model to final20.h5
90/90 [==============================] - 407s 5s/step - loss: 180.0580 - val_loss: 61.9495
Epoch 4/100
90/90 [==============================] - ETA: 0s - loss: 174.0286
Epoch 4: val_loss did not improve from 61.94953
90/90 [===========